In [1]:
require 'nngraph';


function createCModel(getAttScores)
    local image_feat1 = nn.Identity()();
    local image_feat2 = nn.Identity()();
    local question = nn.Identity()();
    local confidence = nn.Identity()();

    local getAttScores1 = getAttScores:clone()
    local getAttScores2 = getAttScores1:clone('weight','bias', 'gradWeight','gradBias')

    local attScores1 = getAttScores1(image_feat1)
    local attScores2 = getAttScores2(image_feat2)

    local conf_pred1 = nn.DotProduct()({attScores1, question})
    local conf_pred2 = nn.DotProduct()({attScores2, question})

    local conf_pred11 = nn.View(-1, 1)(conf_pred1);
    local conf_pred22 = nn.View(-1, 1)(conf_pred2);
    local confidence11 = nn.View(-1, 1)(confidence);

    local input = nn.JoinTable(2)({conf_pred11, conf_pred22, confidence11});

    local y = nn.Sequential()
    y:add(nn.Linear(3, 2))
    y:add(nn.Abs())
    y:add(nn.Linear(2,1))
    y:add(nn.Sigmoid())

    local prob = y(input);

    nngraph.annotateNodes();
    return nn.gModule({image_feat1, image_feat2, question, confidence}, {prob});
end

B_model = torch.load('B_model_nn.t7')
getAttScores = B_model.modules[2]
-- C_model = createCModel(getAttScores)
-- torch.save('C_model.t7', C_model)

In [2]:
B_mo

nn.Sequential {
  [input -> (1) -> (2) -> (3) -> (4) -> output]
  (1): nn.ReLU
  (2): nn.Dropout(0.500000)
  (3): nn.Linear(4096 -> 42)
  (4): nn.Sigmoid
}
{
  gradInput : FloatTensor - empty
  modules : 
    {
      1 : 
        nn.ReLU
        {
          inplace : true
          threshold : 0
          _type : torch.FloatTensor
          output : FloatTensor - empty
          gradInput : FloatTensor - empty
          name : relu7
          val : 0
        }
      2 : 
        nn.Dropout(0.500000)
        {
          v2 : true
          noise : FloatTensor - empty
          _type : torch.FloatTensor
          p : 0.5
          gradInput : FloatTensor - empty
          output : FloatTensor - empty
          train : true
          name : drop7
        }
      3 : 
        nn.Linear(4096 -> 42)
        {
          gradBias : FloatTensor - size: 42
          weight : FloatTensor - size: 42x4096
          _type : torch.FloatTensor
          output : FloatTensor - empty
          gradInput